In [206]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torch
import time

torch.manual_seed(1)

In [186]:
with open(r'C:\Users\nelso\OneDrive\Machine_Learning\Project\tweets.txt', encoding = 'utf8') as f:
    tweets = f.readlines()

del tweets[0]

In [187]:
with open(r'C:\Users\nelso\OneDrive\Machine_Learning\Project\tweets.txt', encoding = 'utf8') as f:
    content=list(set(f.read()))

In [208]:
ix_to_char = {ix:char for ix, char in enumerate(content)}
char_to_ix = {char:ix for ix, char in enumerate(content)}
vocab_size = len(char_to_ix)

In [216]:
def prepare_tweet(tweet, target = False):
    if target==True:
        tweet_ix = torch.tensor([char_to_ix[c] for c in tweet[1:]], dtype=torch.long)
    else:
        tweet_ix = torch.tensor([char_to_ix[c] for c in tweet[:-1]], dtype=torch.long)
    return tweet_ix

In [219]:
tweet = prepare_tweet(tweets[0])

torch.Size([177])

A few notes on building the neural network
- Sequence length should vary for each tweet. Should be length of tweet.
- Need to create an \<EOS> marker
- Embedding happens when putting input tweet into x tensor
- From embedding, x should be passed through LSTM
- Then need a hidden layer which should maybe spit out something the size of vocab?
- Use a softmax to get the probability
- NLLL to get the loss
- Backprop
- Run this through 2 for loops: for epoch and for tweet

In [212]:
class RNN(nn.Module):
    def __init__(self, embed_dim, hidden_dim, vocab_size, n_layers=1):
        super(RNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embed = nn.Embedding(vocab_size, embed_dim)
        
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers = n_layers)
        
        self.hidden2char = nn.Linear(hidden_dim, vocab_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return(torch.zeros(self.n_layers, 1, self.hidden_dim),
              torch.zeros(self.n_layers, 1, self.hidden_dim))
    
    def forward(self, tweet):
        embeds = self.embed(tweet)
        lstm_out, self.hidden = self.lstm(embeds.view(len(tweet), 1, -1),
                                         self.hidden)
        output = self.hidden2char(lstm_out.view(len(tweet), -1))
        log_probs = F.log_softmax(output, dim=1)
        return log_probs
        

In [220]:
hidden_dim = 128
embed_size = 128 # This is going to probably get up to 128 or 256
epochs = 3

model = RNN(embed_size, hidden_dim, vocab_size)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=.1)
start = time.time()
for epoch in range(epochs):
    total_loss = 0
    for tweet in tweets:
        model.zero_grad()
        
        model.hidden = model.init_hidden()
        
        tweet_x = prepare_tweet(tweet)
        tweet_y = prepare_tweet(tweet, target=True)
        
        log_probs = model(tweet_x)
        
        loss = loss_function(log_probs, tweet_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print('Epoch', epoch+1, 'Completed in', time.time()-start,'seconds; Loss:', total_loss)
    print(list(model.parameters())[0])

Epoch 1 Completed in 60.3801064491272 seconds; Loss: 1111.8014950752258
Parameter containing:
tensor([[ 0.9740, -1.2055, -0.5713,  ...,  0.5790,  0.1663,  0.1402],
        [ 1.0315, -2.1694, -2.3597,  ..., -0.5947, -1.2716, -0.8087],
        [-0.5506, -1.0910,  0.0448,  ...,  0.1050,  1.0393, -1.0583],
        ...,
        [-1.1538, -1.2682,  1.8882,  ...,  0.3712, -0.6047, -1.0388],
        [ 2.1208,  0.1075,  0.7270,  ..., -0.2491,  0.3593,  0.3203],
        [-0.3521, -0.0316, -1.6745,  ...,  0.2884,  1.0849,  0.0208]],
       requires_grad=True)
Epoch 2 Completed in 125.79818987846375 seconds; Loss: 918.2247023582458
Parameter containing:
tensor([[ 0.9771, -1.2068, -0.5709,  ...,  0.5805,  0.1667,  0.1400],
        [ 1.0307, -2.1747, -2.3655,  ..., -0.5922, -1.2747, -0.8076],
        [-0.5496, -1.0913,  0.0446,  ...,  0.1048,  1.0395, -1.0582],
        ...,
        [-1.1549, -1.2678,  1.8848,  ...,  0.3716, -0.6054, -1.0370],
        [ 2.1224,  0.1066,  0.7265,  ..., -0.2498,  0.359

In [227]:
def generate_tweet(first):
    char = torch.tensor(char_to_ix[first], dtype = torch.long)
    hidden = model.init_hidden()
    output_tweet = first
    letter = first
    
    #while letter != '/n':
    for i in range(5):
        embeds = embed(char)
        output, hidden = model.lstm(embeds.view(1, 1, -1), hidden)
        topv, topi = output.topk(1)
        topi = topi[0][0][0].item()
        letter = ix_to_char[topi]
        output_tweet += letter
        char = torch.tensor(char_to_ix[letter], dtype = torch.long)
        
    return output_tweet

In [228]:
generate_tweet('a')

KeyError: 89